# Accident Propensity Index Calculation

In [2]:
import pandas as pd
import math

# Define a function to calculate the distance between two points
def distance(point1, point2):
    lat1, lon1 = point1
    lat2, lon2 = point2
    km_per_lat = 110.574 # km per degree latitude
    km_per_lon = 111.320 # km per degree longitude at the equator
    dx = (lon2 - lon1) * km_per_lon * math.cos((lat1 + lat2) / 2)
    dy = (lat2 - lat1) * km_per_lat
    return math.sqrt(dx**2 + dy**2)

# Define a function to calculate the distance between a point and a line segment
def distance_to_segment(point, segment_start, segment_end):
    px, py = point
    x1, y1 = segment_start
    x2, y2 = segment_end
    dx, dy = x2 - x1, y2 - y1
    segment_length_squared = dx*dx + dy*dy
    if segment_length_squared == 0:
        return distance(point, segment_start)
    t = max(0, min(1, ((px - x1) * dx + (py - y1) * dy) / segment_length_squared))
    x = x1 + t * dx
    y = y1 + t * dy
    return distance(point, (x, y))

# Define a function to find accidents on a given route within a maximum distance
def find_accidents_on_route(start_point, end_point, max_distance):
    # Create a mask for accidents that are within the maximum distance from the route
    mask = data.apply(lambda row: distance_to_segment((row['latitude'], row['longitude']), start_point, end_point) <= max_distance, axis=1)

    # Return the accidents that match the mask
    return data.loc[mask]

In [3]:
# Accident data frame
data = pd.read_csv('accident_data.csv')

# Route start and end point
start_point = (33.781544, -84.403656)
end_point = (33.781538, -84.398188)

# Maximal distance of accidents from route in kilometers
max_distance = 0.05

In [4]:
# Run after entering the accident, route, and distance data
accidents = find_accidents_on_route(start_point, end_point, max_distance)
print("Accidents on Route")
print(accidents)
print("\nAccidents off Route")
not_on_route = ~data['id'].isin(accidents['id'])
print(data[not_on_route])

Accidents on Route
   id   latitude  longitude  severity_score on_route
0   1  33.781549 -84.403342               2      yes
2   3  33.781534 -84.401825               4      yes
3   4  33.781574 -84.403317               1      yes
6   7  33.781565 -84.399913               1      yes

Accidents off Route
   id   latitude  longitude  severity_score on_route
1   2  33.786186 -84.401266               3       no
4   5  33.780310 -84.403067               3       no
5   6  33.782688 -84.401343               2       no
